In [ ]:
# Import packages
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
AorC = 'Cathodic'
data_dir = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence\\data\\Fig5'

background_path = os.path.join(data_dir, f'{AorC}.tif')
image = Image.open(background_path).resize((320, 320))
backarr = np.array(image)

axon_colors = {(255, 38, 0, 255),
               (0, 225, 255, 255),
               (229, 255, 0, 255),
               (0, 38, 255, 255),
               (255, 130, 130, 255),
               (0, 255, 106, 255),
               (84, 0, 118, 255),
               (255, 38, 0, 255),
               (0, 225, 255, 255)}

time_folder = os.path.join(data_dir, AorC)
for time_file in [f'{i}.tif' for i in range(1, len(os.listdir(time_folder))+1)]:
    image = Image.open(os.path.join(time_folder, time_file))
    imarr = np.array(image)
    plt.imshow(backarr)
    plt.imshow(imarr, alpha=0.3)
    plt.show()

In [32]:
backarr[50, 10]

array([255,  38,   0, 255], dtype=uint8)